In [1]:
# Data processing
import pandas as pd
import numpy as np
from mizani.breaks import date_breaks
from mizani.formatters import date_format

# Data Viz
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read in the data
monthly_2002 = pd.read_csv("raw/NOAA/monthly_trade_2002.csv")
monthly_2002.shape

(7116, 16)

In [3]:
# Explore the data
monthly_2002.head()

,Year,Month,Product Name,Country Name,Volume (kg),Value (USD),Trade Type,Census Country Code,Edible code,FAO Country Code,HTS Number,RFMO,Trade Associations,US Customs District Code,US Customs District Name,Month number
0,2002,January,CATFISH NSPF FILLET FROZEN,GUYANA,"1,769","5,655",IMP,3120,E,328,304206030,NaN,"CARICOM, CBERA",10,"NEW YORK, NY",1
1,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,"9,979","29,237",IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",18,"TAMPA, FL",1
2,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,"59,239","135,651",IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",20,"NEW ORLEANS, LA",1
3,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,"317,779","849,402",IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",27,"LOS ANGELES, CA",1
4,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,"5,102","10,944",IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",28,"SAN FRANCISCO, CA",1


In [4]:
# Reture the data type for each column
monthly_2002.dtypes

Year                         int64
Month                       object
Product Name                object
Country Name                object
Volume (kg)                 object
Value (USD)                 object
Trade Type                  object
Census Country Code          int64
Edible code                 object
FAO Country Code             int64
HTS Number                   int64
RFMO                        object
Trade Associations          object
US Customs District Code     int64
US Customs District Name    object
Month number                 int64
dtype: object

In [5]:
# Adjust the data types
monthly_2002['Volume (kg)'] = monthly_2002['Volume (kg)'].str.replace(',', '').astype(int)
monthly_2002['Value (USD)'] = monthly_2002['Value (USD)'].str.replace(',', '').astype(int)
monthly_2002.dtypes

Year                         int64
Month                       object
Product Name                object
Country Name                object
Volume (kg)                  int64
Value (USD)                  int64
Trade Type                  object
Census Country Code          int64
Edible code                 object
FAO Country Code             int64
HTS Number                   int64
RFMO                        object
Trade Associations          object
US Customs District Code     int64
US Customs District Name    object
Month number                 int64
dtype: object

In [6]:
# Check unique values of Product Name
sorted(monthly_2002['Product Name'].unique())

['CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA DRIED',
 'CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA FILLET DRIED/SALTED/BRINE',
 'CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA SALTED > 6.8 KG',
 'CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA SALTED NOT > 6.8KG',
 'CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA SMOKED',
 'CARP,CATFISH,EELS,SNAKEHEAD MEAT FRESH',
 'CARP,CATFISH,EELS,SNAKEHEAD,TILAPIA FROZEN NOT > 6.8KG',
 'CARP,CATFISH,EELS,SNAKEHEAD,TILAPIA MEAT FROZEN > 6.8KG',
 'CARP,CATFISH,EELS,SNAKEHEAD,TILAPIA MINCED FROZEN > 6.8KG',
 'CARP,CATFISH,EELS,SNAKEHEAD,TILAPIA SURIMI',
 'CATFISH (ICTALURUS) FILLET FRESH',
 'CATFISH (ICTALURUS) FILLET FROZEN',
 'CATFISH (ICTALURUS) FRESH',
 'CATFISH (ICTALURUS) FROZEN',
 'CATFISH (ICTALURUS) MEAT FRESH',
 'CATFISH (ICTALURUS,PANGASIUS,SILURUS) FRESH NOT > 6.8KG',
 'CATFISH (PANGASIUS) FILLET FRESH',
 'CATFISH (PANGASIUS) FILLET FROZEN',
 'CATFISH (PANGASIUS) FRESH',
 'CATFISH (PANGASIUS) FROZEN',
 'CATFISH (PANGASIUS) MEAT FRESH'

In [7]:
# Remove rows that contain 'CARP,CATFISH,EELS'
monthly_2002 = monthly_2002[~monthly_2002['Product Name'].str.contains('CARP,CATFISH,EELS')]
monthly_2002.shape

(3165, 16)

In [7]:
# Get the indices of 2012 January and February data
index_2012 = monthly_2002[(monthly_2002['Year']==2012)&
                          (monthly_2002['Month number']<=2)].index
  
# Drop the corresponding rows
monthly_2002 = monthly_2002.drop(index_2012).reset_index(drop=True)

# Rename columns
monthly_2002.columns = ['year','month','product_name','country_name','volume_kg',
                           'value_USD','trade_type','census_country_code','edible_code',
                           'FAO_country_code','HTS_number','RFMO','trade_associations',
                           'US_customs_district_code','US_customs_district_name','month_number']

monthly_2002

,year,month,product_name,country_name,volume_kg,value_USD,trade_type,census_country_code,edible_code,FAO_country_code,HTS_number,RFMO,trade_associations,US_customs_district_code,US_customs_district_name,month_number
0,2002,January,CATFISH NSPF FILLET FROZEN,GUYANA,1769,5655,IMP,3120,E,328,304206030,NaN,"CARICOM, CBERA",10,"NEW YORK, NY",1
1,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,9979,29237,IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",18,"TAMPA, FL",1
2,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,59239,135651,IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",20,"NEW ORLEANS, LA",1
3,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,317779,849402,IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",27,"LOS ANGELES, CA",1
4,2002,January,CATFISH NSPF FILLET FROZEN,VIETNAM,5102,10944,IMP,5520,E,704,304206030,NaN,"APEC, ASEAN",28,"SAN FRANCISCO, CA",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,2020,October,"CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA...",SOUTH KOREA,11526,193178,IMP,5800,E,410,305520000,"CCAMLR, IATTC, NAFO","APEC, CCAMLR, IATTC, ICCAT, NAFO, OECD",27,"LOS ANGELES, CA",10
7028,2020,October,"CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA...",GHANA,1841,2450,IMP,7490,E,288,305520000,NaN,ICCAT,10,"NEW YORK, NY",10
7029,2020,October,"CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA...",GHANA,2132,5035,IMP,7490,E,288,305520000,NaN,ICCAT,54,"WASHINGTON, DC",10
7030,2020,October,"CARP,CATFISH,EELS,NILE PERCH,SNAKEHEAD,TILAPIA...",NIGERIA,796,3411,IMP,7530,E,566,305520000,NaN,NaN,10,"NEW YORK, NY",10


In [8]:
# Save the cleaned dataset
monthly_2002.to_csv('output/build/monthly_2002.csv')